In [1]:
import pandas as pd
import json
import random

In [ ]:
#generating standard protocol

name = input("protocol name: ")
parameters = input("parameters: ")
protocol_csv = 'protocol.csv'

In [ ]:
def generate_standard_protocol(input_csv, output_json):
    # Read CSV into a DataFrame
    df = pd.read_csv(input_csv)

    # Initialize the JSON structure
    json_data = {
        "name": name,
        "parameters": [parameters],
        "active": True,
        "classes": [],
        "randomClassOrder": False
    }

    # Iterate through rows in the CSV
    for index, row in df.iterrows():
        # Extract relevant information from the CSV row
        class_name = row['class_name']
        inactive_duration = row['inactive_duration']
        active_duration = row['active_duration']
        preamble = row['preamble']
        epilogue = row['epilogue']
        activeClass = row['activeClass_name']
        repetitions = int(row['repetitions'])

        # Create the class structure
        class_structure = {
            "class": class_name,
            "intervals": [
                {"label": "inactive", "durationInMillis": inactive_duration},
                {"label": activeClass, "durationInMillis": active_duration},
                {"label": "inactive", "durationInMillis": inactive_duration}
            ],
            "repetitions": repetitions,
            "preamble": preamble,
            "epilogue": epilogue
        }

        # Append the class structure to the classes list
        json_data["classes"].append(class_structure)

    # Write the JSON data to a file
    with open(output_json, 'w') as json_file:
        json.dump(json_data, json_file, indent=2)

generate_standard_protocol(protocol_csv, name)

In [2]:
#functions for reaction time
def generate_intervals(total_duration, stimulus_duration, max_isi, min_isi):
    num_intervals = int(total_duration / (stimulus_duration + (max_isi + min_isi) / 2))
    
    print(f"Total Duration: {total_duration} seconds")
    print(f"Stimulus Duration: {stimulus_duration} seconds")
    print(f"Max ISI: {max_isi} seconds")
    print(f"Min ISI: {min_isi} seconds")
    print(f"Number of Intervals: {num_intervals}")

    intervals = []

    for _ in range(num_intervals):
        isi = random.randint(min_isi, max_isi)
        intervals.append(isi)
    
    return intervals

def generate_RT(name, led_color, stim_duration, intensity, unlabeled_intervals, filename="output.json"):
    data = {
        "name": name,
        "parameters": ["test"],
        "active": True,
        "classes": [
            {
                "class": "PMT",
                "intervals": [],
                "type": "REACTION_TIME"
            }
        ],
        "randomClassOrder": True
    }

    class_data = data["classes"][0]

    for i, duration in enumerate(unlabeled_intervals):
        # Unlabeled interval
        class_data["intervals"].append({
            "durationInMillis": duration,
            "ledOptions": {
                "enabled": False
            }
        })

        # STIM interval
        class_data["intervals"].append({
            "label": "STIM",
            "durationInMillis": stim_duration,
            "ledOptions": {
                "color": {
                    "red": led_color[0],
                    "green": led_color[1],
                    "blue": led_color[2]
                },
                "intensity": intensity,
                "enabled": True
            }
        })

    # Set the last interval as an unlabeled interval if the total count is odd
    if len(unlabeled_intervals) % 2 == 1:
        class_data["intervals"].pop()

    with open(filename, "w") as file:
        json.dump(data, file, indent=2)

In [ ]:
# Set the parameters for the reaction time test
name = "example_name"
led_color = [1, 1, 1]
stim_duration = 500 
intensity = 1
total_duration = int(input('Total duration in ms: '))
stimulus_duration = int(input('Stimulus duration in ms: '))
max_isi = int(input('Maximum ISI in ms: '))
min_isi = int(input('Minimum ISI in ms: '))

unlabeled_intervals = generate_intervals(total_duration, stimulus_duration, max_isi, min_isi)
generate_RT(name, led_color, stim_duration, intensity, unlabeled_intervals, filename="output.json")